In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import optimize
import math

In [2]:
USD_EUR = np.array([ 1.0432, 1.0468, 1.0584, 1.0734, 1.0810, 1.0822, 1.0769, 1.0760,\
            1.0781, 1.0940, 1.0940, 1.0597, 1.0693, 1.0804, 1.0716, 1.0837,\
            1.0970, 1.1126, 1.1384, 1.1512, 1.1721, 1.1813, 1.1745, 1.1759,\
            1.1732, 1.1500, 1.1506, 1.1324, 1.1230, 1.1422, 1.1363, 1.1357,\
            1.1297, 1.1040, 1.0916, 1.0922, 1.1201, 1.1260, 1.1480, 1.1651,\
            1.1761, 1.1668, 1.1757, 1.1686, 1.1471, 1.1654, 1.1872, 1.1873,\
            1.2072, 1.2223, 1.2352, 1.2429, 1.2549, 1.2748, 1.2649, 1.2580,\
            1.2531, 1.2567, 1.2756, 1.2739, 1.2528, 1.2274, 1.2297, 1.2273,\
            1.2233])

haz_rate = np.array([0.0010, 0.0009, 0.0006, 0.0007, 0.0010, 0.0014, 0.0022, 0.0003,\
            0.0004, 0.0007, 0.0005, 0.0005, 0.0003, 0.0003, 0.0004, 0.0004,\
            0.0003, 0.0004, 0.0005, 0.0006, 0.0006, 0.0006, 0.0006, 0.0010,\
            0.0010, 0.0009, 0.0008, 0.0013, 0.0016, 0.0017, 0.0012, 0.0018,\
            0.0007, 0.0008, 0.0011, 0.0018, 0.0016, 0.0012, 0.0014, 0.0011,\
            0.0014, 0.0006, 0.0005, 0.0004, 0.0005, 0.0006, 0.0007, 0.0007,\
            0.0012, 0.0005, 0.0005, 0.0023, 0.0050, 0.0030, 0.0063, 0.0080,\
            0.0089, 0.0111, 0.0125, 0.0045, 0.0031, 0.0044, 0.0053, 0.0072,\
            0.0050])

In [3]:
YEAR = 52
IR   = [0.0122, 0.0198, 0.0274, 0.0302, 0.035]
dt = 1/YEAR
QUARTER = YEAR // 4
ALPHA = 0.8
SIGMA = 1.2
h_0   = 0.0083696
R     = 0.4
N_SIM = 10000

In [4]:
IR_curve = [np.exp(-IR[1] * i / YEAR) for i in range(YEAR)] + \
           [np.exp(-IR[1]-IR[2] * i / YEAR) for i in range(YEAR)] + \
           [np.exp(-IR[1]-IR[2]-IR[3] * i / YEAR) for i in range(YEAR)] + \
           [np.exp(-IR[1]-IR[2]-IR[3]-IR[4] * i / YEAR) for i in range(YEAR)]

Z = np.random.normal(size=(4 * YEAR, N_SIM))
SPREADS = [0.0055, 0.005668627, 0.005784314, 0.0059]

In [5]:
def discount(t, max_len):
    return IR_curve[t] if t < max_len else 0

def value(t, T, spread, R):
    n_payments = t // YEAR + 1
    return (1 - R) * discount(t, T) - sum([discount(i * YEAR, T) * spread for i in range(n_payments)])

def simulate(Z, alpha, sigma, h_0, thetas): # thetas = [theta_1, theta_2]
    # we have existing thetas, simulate for one more year
    _, N = Z.shape
    T = len(thetas) * YEAR
    results = np.empty([T, N])
    h_t  = np.full(N, h_0)
    for i, theta in enumerate(thetas):
        for j in range(YEAR):
            t = i * YEAR + j
            results[t, :] = h_t + alpha * (theta - h_t) * dt + sigma * h_t * math.sqrt(dt) * Z[t, :]
            h_t = results[t, :]
    return results

def sim_defaults(results):
    T, N = results.shape
    bern = np.random.binomial(1, results*dt)
    def default_fn(arr):
        out = np.argmax(arr)
        if out == 0 and arr[0] == 0:
            return len(arr)
        else: return out
    return np.apply_along_axis(default_fn, 0, bern), T

def rn_pricing(defaults, T, spread, R):
    return np.array(list(map(lambda t: value(t, T, spread, R), defaults)))

In [6]:
EPS = 1e-6

def bin_search(lo, hi, thetas, spread, it, R):
    it += 1
    guess  = lo + (hi - lo)/2
    results = simulate(Z, ALPHA, SIGMA, h_0, thetas + [guess])
    trials = np.empty(10)
    for i in range(10):
        defaults, T = sim_defaults(results)
        pricing = rn_pricing(defaults, T, spread, R).mean()
        trials[i] = pricing
    pricing = trials.mean()
    if abs(pricing) < EPS or it > 1000:
        return guess
    elif pricing < 0:
        return bin_search(guess, hi, thetas, spread, it, R)
    else:
        return bin_search(lo, guess, thetas, spread, it, R)

In [7]:
# Calibrate 
thetas = []
guess  = 0.
EPS    = 1e-6

for y, spread in enumerate(SPREADS):
    next_theta = bin_search(1e-3, 5e-2, thetas, spread, 0, R)
    print(f"Theta_{y+1} calibrated to be {next_theta}")
    thetas += [next_theta]

Theta_1 calibrated to be 0.011852293096075071
Theta_2 calibrated to be 0.009040766594021893
Theta_3 calibrated to be 0.012600900100544096
Theta_4 calibrated to be 0.010085810910103327


In [11]:
def bin_search_h(lo, hi, thetas, spread, it, R):
    it += 1
    guess  = lo + (hi - lo)/2
    results = simulate(Y, ALPHA, SIGMA, guess, thetas)
    trials = np.empty(10)
    for i in range(10):
        defaults, T = sim_defaults(results)
        pricing = rn_pricing(defaults, T, spread, R).mean()
        trials[i] = pricing
    pricing = trials.mean()
    if abs(pricing) < EPS or it > 1000:
        return guess
    elif pricing < 0:
        return bin_search(guess, hi, thetas, spread, it, R)
    else:
        return bin_search(lo, guess, thetas, spread, it, R)

In [9]:
lock_thetas = thetas

In [42]:
SPREADS_A = [0.0042, 0.0046, 0.0049, 0.005, 0.0051]

In [ ]:
h_s = []
for y, spread in enumerate(SPREADS_A):
    next_h = bin_search_h(1e-3, 5e-2, thetas[:y+1], spread, 0, 0.3)
    print("Next h calibrated to be ", next_h)
    h_s.append(next_h)